## Pipeline обработки запроса

### Загрузка данных и функций (достаточно выполнить 1 раз)

In [ ]:
from utils import setup_environment, load_data
from retriever import retriever
from planner import planner
from dreamer import dreamer
from grounder import grounder
from executor import executor
from schemas import RetrieverOut, PlannerOut, DreamerOut

from openai import OpenAI
from config import PipelineConfig


api_key, db_path = setup_environment()
db = load_data(db_path, wave_filter=["2025-03"])    # Фильтр по последней волне
db = db.sample(1000, random_state=42)               # Ограничение по числу строк - уменьшаем контексты

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

2025-11-14 13:38:43 | INFO     | main                 | Environment loaded: DB_PATH=C:\Users\kateu\Documents\Ivanovs Index\V2\DB\25W3\db_CORRECTED.parquet
2025-11-14 13:38:43 | INFO     | main                 | Loading data from C:\Users\kateu\Documents\Ivanovs Index\V2\DB\25W3\db_CORRECTED.parquet
2025-11-14 13:38:44 | INFO     | main                 | Full dataset shape: (26505646, 5)
2025-11-14 13:38:44 | INFO     | main                 | Filtered by waves: ['2025-03'], new shape: (672405, 5)


### Настройка используемых моделей и их температур

Если нужно поменять модель, это делается здесь

In [2]:
PPL_cfg = PipelineConfig.setup(
    df=db, client=client,
    # параметры ретривера
    retriever_params={
        "model": "deepseek/deepseek-chat-v3.1",
        # "model": "alibaba/tongyi-deepresearch-30b-a3b:free",
        "temperature": 0.8,
        "reasoning_effort": None
    },
    # параметры аналитика-выдумщика
    dreamer_params={
        "model": "deepseek/deepseek-chat-v3.1",
        # "model": "alibaba/tongyi-deepresearch-30b-a3b:free",
        "temperature": 1,
        "reasoning_effort": "high"
    },
    # параметры планировщика
    planner_params={
        "model": "deepseek/deepseek-chat-v3.1",
        # !!! "model": "deepseek/deepseek-chat-v3.1", # Платная модель
        "temperature": 0.2
    }
)

### Еще варианты моделей
# "alibaba/tongyi-deepresearch-30b-a3b:free"
# "meta-llama/llama-4-maverick:free"
# !!! "deepseek/deepseek-chat-v3.1" # Платная модель

### Retriever

Извлекает релевантные вопросы на основе запроса пользователя и всего набора вопросов

Пользовательский запрос

In [3]:
# user_query = "Я хочу посчитать индекс потребительской уверенности по потребителям из москвы"
# user_query = "Я хочу посчитать размер средних сбережений и норму сбережений среди тех, у кого они есть"
# user_query = "Я хочу посчитать долю взаимопроникновения клиентов Чижика и Пятерочки"
# user_query = "Мне нужны все вопросы, связанные с автотранспортом (сроки владения, предпочтения по маркам, планы по покупке и пр.)"
user_query = "Посчитай размер средних сбережений и норму сбережений среди тех, у кого они есть"

Обращение к LLM

> ! Возможны проблемы парсинга ответов

In [4]:
# Сохранение, чтобы не делать 1 и тот же запрос кучу раз:
# можно 1 раз сохранить, а далее только читать

# retriever_out = retriever(user_query, PPL_cfg)
# retriever_out.save("cur_retrieved.json")

Можно посмотреть reasoning, если модель его поддерживает и он был указан в параметрах

In [5]:
# print(retriever_out.reasoning)

In [6]:
retriever_out = RetrieverOut.load("cur_retrieved.json")
print(retriever_out)

1. '[A26] Каков размер денежных сбережений Вашей семьи (наличные деньги, депозиты в банках, ценные бумаги)?'
	Reason: Это прямой вопрос о размере сбережений, который содержит конкретные числовые интервалы (например, '60,000 - 90,000 рублей', '400000 - 700,000 рублей'). Он необходим для расчета среднего размера сбережений среди тех, у кого они есть. Ответ "Не сберегаю" в данном списке ответов отсутствует, что означает, что этот вопрос задается только тем, у кого сбережения есть, либо его ответы можно использовать для фильтрации тех, у кого сбережения отсутствуют (отсутствие низких диапазонов в ответах может указывать на пропуск таких респондентов).
2. '[Family_income_new] К какому из следующих интервалов Вы могли бы отнести месячный доход ВАШЕЙ СЕМЬИ (доход членов семьи, проживающих в одной квартире)?'
	Reason: Для расчета нормы сбережений (сбережения / доход) необходим вопрос о доходе респондента. Этот вопрос предоставляет интервалы месячного дохода семьи ('40.000-49.999 рублей', '100.

### Dreamer

Строит план анализа без привязки к функционалу

In [7]:
# dreamer_out = dreamer(user_query, PPL_cfg)
# dreamer_out.save("cur_dreamed.json")

In [8]:
dreamer_out = DreamerOut.load("cur_dreamed.json")
print(dreamer_out)

####################
АНАЛИЗ
####################
Отлично, задача понятна. На основе предоставленных данных я составлю подробный план анализа для расчета среднего размера сбережений и нормы сбережений среди тех респондентов, у кого они есть.

### План анализа

**Цель:** Рассчитать два ключевых показателя для респондентов, имеющих сбережения:
1.  **Размер средних сбережений**
2.  **Норма сбережений**

---

#### Шаг 1: Определение целевой группы (Фильтрация респондентов)

Первым делом необходимо отфильтровать респондентов, которые **имеют сбережения**.

*   **Источник данных:** Вопрос `[A26] Каков размер денежных сбережений Вашей семьи...`
*   **Действие:** Из всей выборки нужно отобрать только тех респондентов, чьи ответы не являются пустыми и не соответствуют варианту "Нет сбережений" (если такой вариант явно присутствует в полных данных, но в выгрузке он не показан, однако приведенные ответы `['400000 - 700,000 рублей', '60,000 - 90,000 рублей']` явно указывают на тех, у кого сбережени

Можно отдельно посмотреть **анализ** и **reasoning**, раскомментировав соответствующую строку

In [9]:
# print(dreamer_out.analysis)
# print(dreamer_out.reasoning)

### Planner

Строит план на основе пользовательского запроса и набора релевантных ответов, к которым подмешиваются их ответы

В план включаются команды из [`capability_spec.py`](./capability_spec.py)

In [ ]:
# Сохранение, чтобы не делать 1 и тот же запрос кучу раз:
# можно 1 раз сохранить, а далее только читать

# planner_out = planner(user_query, dreamer_out, PPL_cfg)
# planner_out.save("cur_planned.json")

2025-11-14 13:38:45 | INFO     | planner              | Starting planner for query: Посчитай размер средних сбережений и норму сбережений среди тех, у кого они есть...
2025-11-14 13:38:45 | DEBUG    | planner              | Generated prompt of length: 71409
2025-11-14 13:38:45 | DEBUG    | planner              | Calling LLM with model: deepseek/deepseek-chat-v3.1
2025-11-14 13:38:53 | INFO     | httpx                | HTTP Request: POST https://openrouter.ai/api/v1/responses "HTTP/1.1 200 OK"
2025-11-14 13:38:58 | DEBUG    | planner              | Plan received: 4 steps
2025-11-14 13:38:58 | DEBUG    | planner              | Plan analysis: Расчет среднего размера сбережений и нормы сбережений среди респондентов, имеющих сбережения. Для этого сначала фильтруем респондентов с ответами на вопрос [A26], которые явно указывают на наличие сбережений (исключая пустые ответы и вариант 'Нет сбережений', который в данных не представлен, но подразумевается). Затем для отфильтрованной группы рассч

In [11]:
planner_out = PlannerOut.load("cur_planned.json")
print(planner_out)

АНАЛИЗ: Расчет среднего размера сбережений и нормы сбережений среди респондентов, имеющих сбережения. Для этого сначала фильтруем респондентов с ответами на вопрос [A26], которые явно указывают на наличие сбережений (исключая пустые ответы и вариант 'Нет сбережений', который в данных не представлен, но подразумевается). Затем для отфильтрованной группы рассчитываем средний размер сбережений, преобразуя интервалы в середины диапазонов. Норму сбережений рассчитываем на основе вопроса [N4], также преобразуя процентные интервалы в середины диапазонов.
0. [s1] OperationType.LOAD_DATA
	Goal: Загрузить данные опроса за март 2025 года
	Inputs: {'waves': ['2025-03']}
	Outputs: ['dataset']
	Constraints: {}
	Depends on: []
1. [s2] OperationType.FILTER
	Goal: Отфильтровать респондентов, у которых есть сбережения (ответивших на вопрос [A26] с непустыми значениями)
	Inputs: {'dataset': 'dataset', 'question': '[A26] Каков размер денежных сбережений Вашей семьи (наличные деньги, депозиты в банках, цен

### Grounder

Привязка шагов плана к имеющимся функциям (из [`operations.py`](./operations.py))

In [12]:
grounder_out = grounder(planner_out)

2025-11-14 13:39:32 | INFO     | grounder             | Starting grounder
2025-11-14 13:39:32 | DEBUG    | grounder             | Grounding step 1/4: s1
2025-11-14 13:39:32 | DEBUG    | grounder             | Successfully grounded step s1: LOAD_DATA
2025-11-14 13:39:32 | DEBUG    | grounder             | Grounding step 2/4: s2
2025-11-14 13:39:32 | DEBUG    | grounder             | Successfully grounded step s2: FILTER
2025-11-14 13:39:32 | DEBUG    | grounder             | Grounding step 3/4: s3
2025-11-14 13:39:32 | DEBUG    | grounder             | Successfully grounded step s3: PIVOT
2025-11-14 13:39:32 | DEBUG    | grounder             | Grounding step 4/4: s4
2025-11-14 13:39:32 | DEBUG    | grounder             | Successfully grounded step s4: PIVOT
2025-11-14 13:39:32 | INFO     | grounder             | Grounder completed: 4 steps grounded


### Executor

Валидирует план, осуществляет топологическую сортировку, выполняет последовательность шагов

In [13]:
ctx = {"dataset": db}
ctx = executor(grounder_out, ctx)

2025-11-14 13:39:34 | INFO     | executor             | Starting executor with 4 steps
2025-11-14 13:39:34 | DEBUG    | executor             | Validating step dependencies
2025-11-14 13:39:34 | DEBUG    | executor             | Performing topological sort
2025-11-14 13:39:34 | DEBUG    | executor             | Topological sort completed: ['s1', 's2', 's3', 's4']
2025-11-14 13:39:34 | DEBUG    | executor             | Dependencies validation passed
2025-11-14 13:39:34 | DEBUG    | executor             | Performing topological sort
2025-11-14 13:39:34 | DEBUG    | executor             | Topological sort completed: ['s1', 's2', 's3', 's4']
2025-11-14 13:39:34 | INFO     | executor             | Steps execution order: ['s1', 's2', 's3', 's4']
2025-11-14 13:39:34 | INFO     | executor             | Executing step 1/4: s1 (LOAD_DATA)
2025-11-14 13:39:34 | DEBUG    | executor             |   waves = ['2025-03'] (literal)
2025-11-14 13:39:34 | DEBUG    | executor             |   dataset = ctx[

Далее можно извлекать созданные таблицы из `ctx`

In [14]:
opts = [f"{i}. '{n}'" for i, n in enumerate(ctx.keys(), start=1)]
opts = "\n".join(opts)

print(f"Доступные варианты:\n{opts}")

Доступные варианты:
1. 'dataset'
2. 'filtered_dataset'
3. 'savers_dataset'
4. 'pivot'
5. 'savings_amount_pivot'
6. 'savings_rate_pivot'


Пока что извлекать и смотреть можно только ручками...

In [ ]:
ctx["savings_rate_pivot"]